# Sprint 13: Introducción al proyecto

Bienvenido al **Sprint 13** del proyecto **Sure Tomorrow**.  
En este Sprint se presenta el contexto general y los objetivos del proyecto, así como una visión global de las cuatro tareas que resolveremos:

- **Tarea 1:** Encontrar clientes similares (kNN) para campañas de marketing.  
- **Tarea 2:** Clasificación binaria para estimar quién recibirá al menos un beneficio.  
- **Tarea 3:** Regresión lineal desde cero para predecir el número de prestaciones.  
- **Tarea 4:** Ofuscación reversible de datos personales para demostrar privacidad con auditoría.

## Carga de datos e imports

In [12]:
# Imports y carga de datos
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
import math

# Carga del dataset (debe estar en la misma carpeta que el notebook)
df = pd.read_csv('insurance_us.csv')
df = df.rename(columns={
    'Gender': 'gender',
    'Age': 'age',
    'Salary': 'income',
    'Family members': 'family_members',
    'Insurance Benefit Count': 'insurance_benefits'
})

df.head()

,gender,age,income,family_members,Insurance benefits
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df['Insurance benefits'].unique()

array([0, 1, 2, 3, 5, 4], dtype=int64)

### Tarea 1: ¿Para qué sirve?
En esta primera tarea usamos k-Nearest Neighbors para identificar los *k* clientes más parecidos a uno dado.
**¿Qué buscamos?**  
- Agrupar usuarios con características similares: género, edad, ingresos y tamaño de familia.  
- Apoyar campañas de marketing personalizadas mostrando ofertas a perfiles afines.

## Tarea 1: Clientes Similares

In [16]:
# Preparación de atributos
feature_names = ['gender', 'age', 'income', 'family_members']

In [17]:
df_knn = df[feature_names].copy()
#df_knn['gender'] = df_knn['gender'].map({'Male': 0, 'Female': 1})

In [18]:
df_knn.head()

,gender,age,income,family_members
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


In [27]:
# Función para obtener k vecinos
def get_knn(data, idx, k=10, metric='euclidean'):
    nbrs = NearestNeighbors(n_neighbors=k+1, metric=metric)
    nbrs.fit(data)
    distances, indices = nbrs.kneighbors(data)
    return indices[idx][1:], distances[idx][1:]

# Ejemplo de uso
ids, dist = get_knn(df_knn, idx=0, k=5, metric='euclidean')
print('Vecinos:', ids, 'Distancias:', np.round(dist, 2))

Vecinos: [2022 1225 4031 3424  815] Distancias: [1.   1.73 3.16 3.32 4.12]


### Tarea 2: ¿Para qué sirve?
Problema de clasificación binaria: predecir la probabilidad de que un cliente reciba al menos una prestación.
**¿Qué buscamos?**  
- Estimar el riesgo de que un cliente haga uso del seguro.  
- Ayudar a la compañía a ajustar primas y diseñar campañas de retención.

## Tarea 2: Clasificación Binaria

In [30]:
df.columns

Index(['gender', 'age', 'income', 'family_members', 'Insurance benefits'], dtype='object')

In [31]:
# Definición de la variable objetivo
df['received_benefit'] = (df['Insurance benefits'] > 0).astype(int)
print(df['received_benefit'].value_counts(normalize=True))

received_benefit
0    0.8872
1    0.1128
Name: proportion, dtype: float64


In [32]:
df.head()

,gender,age,income,family_members,Insurance benefits,received_benefit
0,1,41.0,49600.0,1,0,0
1,0,46.0,38000.0,1,1,1
2,0,29.0,21000.0,0,0,0
3,0,21.0,41700.0,2,0,0
4,1,28.0,26100.0,0,0,0


In [33]:

# División de datos en entrenamiento y prueba
X = df_knn.values
y = df['received_benefit'].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Modelo kNN
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn.fit(X_train, y_train)
y_prob_knn = clf_knn.predict_proba(X_test)[:,1]
print('kNN -> Accuracy:', accuracy_score(y_test, clf_knn.predict(X_test)),  'AUC:', roc_auc_score(y_test, y_prob_knn))

# Modelo dummy
clf_dummy = DummyClassifier(strategy='prior')
clf_dummy.fit(X_train, y_train)
y_prob_dummy = clf_dummy.predict_proba(X_test)[:,1]
print('Dummy -> Accuracy:', accuracy_score(y_test, clf_dummy.predict(X_test)),  'AUC:', roc_auc_score(y_test, y_prob_dummy))

kNN -> Accuracy: 0.9 AUC: 0.7766438901213217
Dummy -> Accuracy: 0.8873333333333333 AUC: 0.5


### Tarea 3: ¿Para qué sirve?
Implementamos una regresión lineal *“desde cero”* para predecir cuántas prestaciones tendrá un cliente.
**¿Qué buscamos?**  
- Entender la relación lineal entre características y el número de beneficios.  
- Obtener un modelo transparente.

## Tarea 3: Regresión Lineal desde Cero

In [ ]:
# Implementación de la regresión lineal
class MyLinearRegression:
    def fit(self, X, y):
        XtX_inv = np.linalg.inv(X.T.dot(X))
        self.coef_ = XtX_inv.dot(X.T).dot(y)
    def predict(self, X):
        return X.dot(self.coef_)

# Preparación de datos
y_reg = df['insurance_benefits'].values
X_reg = np.hstack([np.ones((len(df), 1)), df_knn.values])
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# Entrenamiento y evaluación
model = MyLinearRegression()
model.fit(Xr_train, yr_train)
y_pred = model.predict(Xr_test)
rmse = math.sqrt(mean_squared_error(yr_test, y_pred))
print(f'RMSE: {rmse:.3f}')

### Tarea 4: ¿Para qué sirve?
Ofuscación de datos personales de forma reversible usando matrices invertibles.
**¿Qué buscamos?**  
- Proteger la privacidad durante el análisis.  
- Poder recuperar los datos originales para auditoría.

## Tarea 4: Ofuscación de Datos

In [34]:
# Ofuscación reversible
X = df_knn.values
rng = np.random.default_rng(42)

In [35]:
rng

Generator(PCG64) at 0x1760A67A180

In [ ]:
while True:
    P = rng.random((X.shape[1], X.shape[1]))
    if np.linalg.det(P) != 0:
        break
X_prime = X.dot(P)
X_recov = X_prime.dot(np.linalg.inv(P))
print('Recuperación exacta:', np.allclose(X, X_recov))

## Conclusiones
- **kNN y escalado:** El escalado balancea las distancias.  
- **Clasificación:** kNN frente a dummy muestra si hay señal.  
- **Regresión:** RMSE y coeficientes revelan la calidad del ajuste.  
- **Ofuscación:** Multiplicación por matriz invertible protege y recupera datos.